TODO: Make sure to use grid search across all models instead of using a test score on multiple different models because this can create data leakage
# Analysis of Fake News in dataset fake_news.csv

Data from: https://www.kaggle.com/datasets/emineyetm/fake-news-detection-datasets

Features:
- `subject`: News Subject
- `Date`
- `Title`
- `Text`

Labels:
- `Label`: 1 if fake, 0 if real

## Setup & Loading

In [17]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'jupyter_black'

In [8]:
from lib import *
from NLP_algorithms import *

In [9]:
df = pd.read_csv("data/fake_news.csv")
display(df.head())
display(df.describe())

,title,text,subject,date,label
0,WOW! LOU DOBBS BLASTS #FakePresident OBAMA: “S...,Lou Dobbs went off on #FakePresident Barack Ob...,politics,2017-07-11,0
1,President WATCH: FUNNIEST Done ‘Couch Commande...,For his final White House Correspondent s Dinn...,News,2016-04-30,0
2,RELIGION OF PROGRESSIVISM: Meet Obama’s NEW Tr...,The religion of Progressivism is working overt...,politics,2016-05-17,0
3,Breitbart Caught Praising Melania For Exact T...,In a stunning example of just how morally bank...,News,2017-05-21,0
4,HOST SICKENING! MOCKS Jeff Grandchildren Senat...,,politics,2017-01-10,0


,label
count,2500.00000
mean,0.47720
std,0.49958
min,0.00000
25%,0.00000
50%,0.00000
75%,1.00000
max,1.00000


## Data Selection & Holdout

In [10]:
X = df["text"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=grs)

# Exploratory Data Analysis

# Model 1: Simple Bag of Words Sentiment Analysis

In [ ]:
estimator_sbw = get_textvec_pipeline(tf_idf = False, pca = False)
grid_search_sbw = GridSearchCV(
    estimator_sbw, 
    get_textvect_grid_search_params(tf_idf= False, pca = False), 
    scoring = "f1", 
    cv = 5, 
    n_jobs = -1
)

In [12]:
grid_search_sbw.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('countvectorizer',
                                        CountVectorizer(min_df=0.01,
                                                        ngram_range=(1, 2),
                                                        stop_words='english')),
                                       ('randomforestclassifier',
                                        RandomForestClassifier(max_depth=10,
                                                               n_jobs=-1,
                                                               random_state=42))]),
             n_jobs=-1,
             param_grid={'countvectorizer__min_df': [0.01, 0.05],
                         'countvectorizer__ngram_range': [(1, 1), (1, 2)]},
             scoring='f1')

In [13]:
grid_search_report("Bag of words sentiment anlysis, linear regression", "F1", grid_search_sbw, X_test, y_test)

Report for:  Bag of words sentiment anlysis, linear regression
Linear SVM Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       252
           1       0.99      0.99      0.99       248

    accuracy                           0.99       500
   macro avg       0.99      0.99      0.99       500
weighted avg       0.99      0.99      0.99       500

Confusion Matrix:
[[249   3]
 [  2 246]]
Best Parameters: {'countvectorizer__min_df': 0.05, 'countvectorizer__ngram_range': (1, 1)}
F1 Score of Best Model: 99.0 %


# Model 2: TF-IDF Sentiment Analysis

In [ ]:
estimator_tfidf = get_textvec_pipeline(tf_idf = True, pca = False)
grid_search_tfidf = GridSearchCV(
    estimator_tfidf, 
    get_textvec_grid_search_params(tf_idf= True, pca = False), 
    scoring = "f1", 
    cv = 5, 
    n_jobs = -1
)

In [15]:
grid_search_tfidf.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidfvectorizer',
                                        TfidfVectorizer(min_df=0.01,
                                                        ngram_range=(1, 2),
                                                        stop_words='english')),
                                       ('randomforestclassifier',
                                        RandomForestClassifier(max_depth=10,
                                                               n_jobs=-1,
                                                               random_state=42))]),
             n_jobs=-1,
             param_grid={'tfidfvectorizer__max_df': [0.9, 1.0],
                         'tfidfvectorizer__min_df': [0.01, 0.05],
                         'tfidfvectorizer__ngram_range': [(1, 1), (1, 2)]},
             scoring='f1')

In [16]:
grid_search_report("TF-IDF sentiment anlysis, linear regression", "F1", grid_search_tfidf, X_test, y_test)

Report for:  TF-IDF sentiment anlysis, linear regression
Linear SVM Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       252
           1       0.99      0.99      0.99       248

    accuracy                           0.99       500
   macro avg       0.99      0.99      0.99       500
weighted avg       0.99      0.99      0.99       500

Confusion Matrix:
[[249   3]
 [  2 246]]
Best Parameters: {'tfidfvectorizer__max_df': 0.9, 'tfidfvectorizer__min_df': 0.05, 'tfidfvectorizer__ngram_range': (1, 1)}
F1 Score of Best Model: 99.05 %
